<a href="https://colab.research.google.com/github/mvish7/PyTorch/blob/master/NN_in_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [0]:
trainset=torchvision.datasets.FashionMNIST(root='./data/FashionMNIST',
                                           train=True, 
                                           download=True,
               transform=transforms.Compose([transforms.ToTensor()]))

In [0]:
class Network(nn.Module):
  def __init__(self):
    super(Network,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
    
    self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
    self.fc2 = nn.Linear(in_features=120, out_features=60)
    self.out = nn.Linear(in_features=60, out_features=10)
    
  def forward(self,t):
    #implementing fwd pass
    
    # input layer
    t=t #returning the tensor as it as, no need of processing
    
    # hidden layer 1 
    t=self.conv1(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    
    #hidden layer 2
    t=self.conv2(t)
    t=F.relu(t)
    t=F.max_pool2d(t, kernel_size=2, stride=2)
    
    #hidden layer 3
    t=t.reshape(-1,12*4*4)  #doing the flattening operation before passing the tensor from conv layer to the linear layer
    t=self.fc1(t)
    t=F.relu(t)
    
    #hidden layer 4
    t=self.fc2(t)
    t=F.relu(t)
    
    #output layer
    t=self.out(t)
    #t=F.softmax(t, dim=1)  no need to use softmax here, coz we are using cross entropy as our loss function and it has built in softmax.
    
    
    return t
  
      

In [0]:
network=Network()

fetching one image from training set

In [0]:
sample_img=next(iter(trainset))

In [0]:
image, label=sample_img
image.shape

as torch requires the ip tensor in the form of [batch_size, ip channels, W, H] we have to change the shape of this tensor by unsqueez operation

In [0]:
image.unsqueeze(0).shape #now we can send this in.

let's get a rough prediction from image (forward pass)

In [0]:
pred = network(image.unsqueeze(0))

In [0]:
pred.shape


In [0]:
pred

In [0]:
label

getting the probabilities of every prediction

In [0]:
F.softmax(pred, dim=1) #why dim=1? last axis of the temsor has the numbers other axes has the arrays